**About Book Crossing Dataset**<br>

This dataset has been compiled by Cai-Nicolas Ziegler in 2004, and it comprises of three tables for users, books and ratings. Explicit ratings are expressed on a scale from 1-10 (higher values denoting higher appreciation) and implicit rating is expressed by 0.

Reference: http://www2.informatik.uni-freiburg.de/~cziegler/BX/ 

**Objective**

This project entails building a Book Recommender System for users based on user-based and item-based collaborative filtering approaches.

#### Execute the below cell to load the datasets

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import pairwise_distances
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import re
import seaborn as sns
from scipy.sparse.linalg import svds

In [2]:
#Loading data
books = pd.read_csv("books.csv", sep=";", error_bad_lines=False, encoding="latin-1")
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']

users = pd.read_csv('users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
users.columns = ['userID', 'Location', 'Age']

ratings = pd.read_csv('ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'bookRating']

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


### Check no.of records and features given in each dataset

In [3]:
print(books.shape)

(271360, 8)


In [4]:
print(users.shape)

(278858, 3)


In [5]:
print(ratings.shape)

(1149780, 3)


## Exploring books dataset

In [6]:
books.head()

ISBN                                          bookTitle  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

             bookAuthor yearOfPublication                   publisher  \
0    Mark P. O. Morford              2002     Oxford University Press   
1  Richard Bruce Wright              2001       HarperFlamingo Canada   
2          Carlo D'Este              1991             HarperPerennial   
3      Gina Bari Kolata              1999        Farrar Straus Giroux   
4       E. J. W. Barber              1999  W. W. Norton &amp; Company   

                                           imageUrlS  \
0  http://images.amazon.com/images/P/0195153448.0...   
1  http://images.amazon.com/images/P/0002005018.0...   
2  http://images.amazon.com/images/P/0060973129.0...   
3  http://images.amazon.com/images/P/0374157065.0...   
4  http://images.amazon.com/images/P/0393045218.0...   

                                           imageUrlM  \
0  http://images.amazon.com/images/P/0195153448.0...   
1  http://images.amazon.com/images/P/0002005018.0...   
2  http://images.amazon.com/images/P/0060973129.0...   
3  http://images.amazon.com/images/P/0374157065.0...   
4  http://images.amazon.com/images/P/0393045218.0...   

                                           imageUrlL  
0  http://images.amazon.com/images/P/0195153448.0...  
1  http://images.amazon.com/images/P/0002005018.0...  
2  http://images.amazon.com/images/P/0060973129.0...  
3  http://images.amazon.com/images/P/0374157065.0...  
4  http://images.amazon.com/images/P/0393045218.0...

### Drop last three columns containing image URLs which will not be required for analysis

In [7]:
books.drop(['imageUrlS', 'imageUrlM', 'imageUrlL'],axis=1,inplace=True)

In [8]:
books.head()

ISBN                                          bookTitle  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

             bookAuthor yearOfPublication                   publisher  
0    Mark P. O. Morford              2002     Oxford University Press  
1  Richard Bruce Wright              2001       HarperFlamingo Canada  
2          Carlo D'Este              1991             HarperPerennial  
3      Gina Bari Kolata              1999        Farrar Straus Giroux  
4       E. J. W. Barber              1999  W. W. Norton &amp; Company

**yearOfPublication**

### Check unique values of yearOfPublication


In [9]:
books.yearOfPublication.unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, '2000', '1995', '1999', '2004',
       '2003', '1990', '1994', '1986', '1989', '2002', '1981', '1993',
       '1983', '1982', '1976', '1991', '1977', '1998', '1992', '1996',
       '0', '1997', '2001', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980'

As it can be seen from above that there are some incorrect entries in this field. It looks like Publisher names 'DK Publishing Inc' and 'Gallimard' have been incorrectly loaded as yearOfPublication in dataset due to some errors in csv file.


Also some of the entries are strings and same years have been entered as numbers in some places. We will try to fix these things in the coming questions.

### Check the rows having 'DK Publishing Inc' as yearOfPublication

In [10]:
books.loc[books.yearOfPublication == 'DK Publishing Inc']

ISBN                                          bookTitle  \
209538  078946697X  DK Readers: Creating the X-Men, How It All Beg...   
221678  0789466953  DK Readers: Creating the X-Men, How Comic Book...   

       bookAuthor  yearOfPublication  \
209538       2000  DK Publishing Inc   
221678       2000  DK Publishing Inc   

                                                publisher  
209538  http://images.amazon.com/images/P/078946697X.0...  
221678  http://images.amazon.com/images/P/0789466953.0...

### Drop the rows having `'DK Publishing Inc'` and `'Gallimard'` as `yearOfPublication`

In [11]:
books = books[(books.yearOfPublication != 'DK Publishing Inc')]

books = books[(books.yearOfPublication != 'Gallimard')]

### Change the datatype of yearOfPublication to 'int'

In [12]:
books.yearOfPublication = books.yearOfPublication.astype('int64')

In [13]:
books.dtypes

ISBN                 object
bookTitle            object
bookAuthor           object
yearOfPublication     int64
publisher            object
dtype: object

### Drop NaNs in `'publisher'` column


In [14]:
books = books.dropna(subset=['publisher'])

books.publisher.isnull().sum()

0

## Exploring Users dataset

In [15]:
print(users.shape)

users.head()

(278858, 3)


userID                            Location   Age
0       1                  nyc, new york, usa   NaN
1       2           stockton, california, usa  18.0
2       3     moscow, yukon territory, russia   NaN
3       4           porto, v.n.gaia, portugal  17.0
4       5  farnborough, hants, united kingdom   NaN

### Get all unique values in ascending order for column `Age`

In [16]:
age=users.Age.unique()

print(sorted(age))

[nan, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 113.0, 114.0, 115.0, 116.0, 118.0, 119.0, 123.0, 124.0, 127.0, 128.0, 132.0, 133.0, 136.0, 137.0, 138.0, 140.0, 141.0, 143.0, 146.0, 147.0, 148.0, 151.0, 152.0, 156.0, 157.0, 159.0, 162.0, 168.0, 172.0, 175.0, 183.0, 186.0, 189.0, 199.0, 200.0, 201.0, 204.0, 207.0, 208.0, 209.0, 210.0, 212.0, 219.0, 220.0, 223.0, 226.0

Age column has some invalid entries like nan, 0 and very high values like 100 and above

### Values below 5 and above 90 do not make much sense for our book rating case...hence replace these by NaNs

In [17]:
users.loc[(users.Age > 90) | (users.Age < 5), 'Age'] = np.nan

### Replace null values in column `Age` with mean

In [18]:
users['Age']=users['Age'].fillna(users['Age'].mean())

users.Age.isnull().sum()

0

### Change the datatype of `Age` to `int`

In [19]:
users.Age=users.Age.astype('int64')

In [20]:
print(sorted(users.Age.unique()))

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]


## Exploring the Ratings Dataset

### check the shape

In [21]:
ratings.shape

(1149780, 3)

In [22]:
n_users = users.shape[0]

n_books = books.shape[0]

In [23]:
ratings.head(5)

userID        ISBN  bookRating
0  276725  034545104X           0
1  276726  0155061224           5
2  276727  0446520802           0
3  276729  052165615X           3
4  276729  0521795028           6

### Ratings dataset should have books only which exist in our books dataset. Drop the remaining rows

In [24]:
ratings_new = ratings[ratings.ISBN.isin(books.ISBN)]

### Ratings dataset should have ratings from users which exist in users dataset. Drop the remaining rows

In [25]:
ratings_new = ratings_new[ratings_new.userID.isin(users.userID)]

### Consider only ratings from 1-10 and leave 0s in column `bookRating`

In [26]:
ratings_corrected = ratings_new[ratings_new.bookRating != 0]

### Find out which rating has been given highest number of times

In [27]:
ratings_corrected['bookRating'].value_counts()

8     91804
10    71225
7     66401
9     60776
5     45355
6     31687
4      7617
3      5118
2      2375
1      1481
Name: bookRating, dtype: int64

### **Collaborative Filtering Based Recommendation Systems**

### For more accurate results only consider users who have rated atleast 100 books

In [28]:
rate_count= ratings_corrected['userID'].value_counts()

ratings_corrected = ratings_corrected[ratings_corrected['userID'].isin(rate_count[rate_count >= 100].index)]

### Generating ratings matrix from explicit ratings


#### Note: since NaNs cannot be handled by training algorithms, replace these by 0, which indicates absence of ratings

In [29]:
ratings_matrix = ratings_corrected.pivot(index='userID', columns='ISBN', values='bookRating').fillna(0)
userID = ratings_matrix.index
ISBN = ratings_matrix.columns
print(ratings_matrix.shape)
ratings_matrix.head()

(449, 66572)


ISBN    0000913154  0001046438  000104687X  0001047213  0001047973  \
userID                                                               
2033           0.0         0.0         0.0         0.0         0.0   
2110           0.0         0.0         0.0         0.0         0.0   
2276           0.0         0.0         0.0         0.0         0.0   
4017           0.0         0.0         0.0         0.0         0.0   
4385           0.0         0.0         0.0         0.0         0.0   

ISBN    000104799X  0001048082  0001053736  0001053744  0001055607  ...  \
userID                                                              ...   
2033           0.0         0.0         0.0         0.0         0.0  ...   
2110           0.0         0.0         0.0         0.0         0.0  ...   
2276           0.0         0.0         0.0         0.0         0.0  ...   
4017           0.0         0.0         0.0         0.0         0.0  ...   
4385           0.0         0.0         0.0         0.0         0.0  ...   

ISBN    B000092Q0A  B00009EF82  B00009NDAN  B0000DYXID  B0000T6KHI  \
userID                                                               
2033           0.0         0.0         0.0         0.0         0.0   
2110           0.0         0.0         0.0         0.0         0.0   
2276           0.0         0.0         0.0         0.0         0.0   
4017           0.0         0.0         0.0         0.0         0.0   
4385           0.0         0.0         0.0         0.0         0.0   

ISBN    B0000VZEJQ  B0000X8HIE  B00013AX9E  B0001I1KOG  B000234N3A  
userID                                                              
2033           0.0         0.0         0.0         0.0         0.0  
2110           0.0         0.0         0.0         0.0         0.0  
2276           0.0         0.0         0.0         0.0         0.0  
4017           0.0         0.0         0.0         0.0         0.0  
4385           0.0         0.0         0.0         0.0         0.0  

[5 rows x 66572 columns]

### Generate the predicted ratings using SVD with no.of singular values to be 50

In [30]:
U, sigma, Vt = svds(ratings_matrix, k = 50)
sigma = np.diag(sigma)
predicted= np.dot(np.dot(U, sigma), Vt) 
pred= pd.DataFrame(predicted, columns = ratings_matrix.columns)
pred.head(10)

ISBN  0000913154  0001046438  000104687X  0001047213  0001047973  000104799X  \
0       0.025341   -0.002146   -0.001431   -0.002146   -0.002146    0.002971   
1      -0.010012   -0.003669   -0.002446   -0.003669   -0.003669    0.001075   
2      -0.015054   -0.015457   -0.010304   -0.015457   -0.015457    0.007281   
3      -0.021499    0.035602    0.023735    0.035602    0.035602    0.030307   
4       0.002077   -0.007965   -0.005310   -0.007965   -0.007965    0.002947   
5      -0.002046    0.018614    0.012409    0.018614    0.018614    0.007966   
6      -0.015920    0.020221    0.013481    0.020221    0.020221    0.014194   
7      -0.010875   -0.010051   -0.006701   -0.010051   -0.010051    0.023373   
8       0.040930   -0.030352   -0.020235   -0.030352   -0.030352    0.018473   
9       0.023473   -0.004168   -0.002778   -0.004168   -0.004168    0.054426   

ISBN  0001048082  0001053736  0001053744  0001055607  ...  B000092Q0A  \
0      -0.003920    0.007035    0.007035    0.012316  ...    0.000180   
1       0.001440   -0.003500   -0.003500    0.001612  ...   -0.000363   
2      -0.014033    0.011941    0.011941    0.011796  ...   -0.000455   
3       0.024215   -0.001053   -0.001053    0.067579  ...    0.002971   
4       0.003057    0.000231    0.000231    0.006080  ...    0.002120   
5       0.022983   -0.005167   -0.005167    0.018298  ...    0.002278   
6       0.016776   -0.002339   -0.002339    0.031493  ...    0.002596   
7      -0.014173    0.017087    0.017087    0.057111  ...    0.000141   
8      -0.031587   -0.013278   -0.013278    0.037860  ...    0.002069   
9      -0.015698   -0.011736   -0.011736    0.114924  ...    0.006618   

ISBN  B00009EF82  B00009NDAN  B0000DYXID  B0000T6KHI  B0000VZEJQ  B0000X8HIE  \
0       0.000226    0.042081   -0.016804   -0.080028    0.004746    0.028314   
1       0.000403    0.008142    0.001104   -0.029224    0.000999    0.002363   
2       0.001907    0.047982    0.005737    0.117859    0.006945    0.003119   
3       0.009912    0.086248   -0.008818    0.016154    0.028848   -0.000125   
4       0.001597   -0.012181    0.009420    0.673459    0.002591   -0.008229   
5       0.003343    0.029729   -0.013429   -0.069757    0.008082    0.003330   
6       0.003667    0.030025    0.012455    0.030068    0.013586    0.001308   
7       0.006781    0.121680   -0.000650   -0.319179    0.018844    0.045594   
8       0.005255    0.070450    0.007803   -0.077213    0.014357    0.005920   
9       0.012665    0.172246   -0.010902   -0.175556    0.049872    0.012847   

ISBN  B00013AX9E  B0001I1KOG  B000234N3A  
0       0.000120   -0.001693    0.067503  
1      -0.000242    0.000029   -0.013059  
2      -0.000304    0.009009   -0.057692  
3       0.001981    0.031201   -0.046664  
4       0.001413    0.004918    0.047773  
5       0.001519    0.008519    0.072081  
6       0.001731    0.014216   -0.004550  
7       0.000094    0.014902    0.012477  
8       0.001380    0.016526   -0.027260  
9       0.004412    0.051858   -0.023768  

[10 rows x 66572 columns]

### Take a particular user_id

### Lets find the recommendations for user with id `2110`

#### Note: Execute the below cells to get the variables loaded

In [31]:
userID = 2110

In [32]:
user_id = 2 #2nd row in ratings matrix and predicted matrix

### Get the predicted ratings for userID `2110` and sort them in descending order

In [33]:
sorted_predictions = pred.iloc[user_id].sort_values(ascending=False)

print(sorted_predictions)

ISBN
0316666343    1.015397
059035342X    0.778665
0345350499    0.697309
0440214041    0.665439
044021145X    0.663549
                ...   
0380709562   -0.162346
0553213164   -0.173301
0440219078   -0.174497
0807508527   -0.204146
0743235150   -0.209452
Name: 2, Length: 66572, dtype: float64


### Create a dataframe with name `user_data` containing userID `2110` explicitly interacted books

In [34]:
user_data = ratings_corrected[ratings_corrected.userID == (userID)]

In [35]:
user_data.head()

userID        ISBN  bookRating
14448    2110  0060987529           7
14449    2110  0064472779           8
14450    2110  0140022651          10
14452    2110  0142302163           8
14453    2110  0151008116           5

In [36]:
user_data.shape

(103, 3)

### Combine the user_data and and corresponding book data(`book_data`) in a single dataframe with name `user_full_info`

In [37]:
book_data = books[books.ISBN.isin(user_data.ISBN)]

In [38]:
book_data.shape

(103, 5)

In [39]:
book_data.head()

ISBN                                          bookTitle  \
246   0151008116                                         Life of Pi   
904   015216250X  So You Want to Be a Wizard: The First Book in ...   
1000  0064472779                                  All-American Girl   
1302  0345307674                     Return of the Jedi (Star Wars)   
1472  0671527215                  Hitchhikers's Guide to the Galaxy   

         bookAuthor  yearOfPublication           publisher  
246     Yann Martel               2002            Harcourt  
904     Diane Duane               2001  Magic Carpet Books  
1000      Meg Cabot               2003        HarperTrophy  
1302     James Kahn               1983       Del Rey Books  
1472  Douglas Adams               1984              Pocket

In [40]:
user_full_info = user_data.merge(book_data)

In [41]:
user_full_info.head()

userID        ISBN  bookRating  \
0    2110  0060987529           7   
1    2110  0064472779           8   
2    2110  0140022651          10   
3    2110  0142302163           8   
4    2110  0151008116           5   

                                     bookTitle       bookAuthor  \
0  Confessions of an Ugly Stepsister : A Novel  Gregory Maguire   
1                            All-American Girl        Meg Cabot   
2           Journey to the Center of the Earth      Jules Verne   
3                             The Ghost Sitter  Peni R. Griffin   
4                                   Life of Pi      Yann Martel   

   yearOfPublication      publisher  
0               2000    Regan Books  
1               2003   HarperTrophy  
2               1965  Penguin Books  
3               2002   Puffin Books  
4               2002       Harcourt

In [42]:
user_full_info.shape

(103, 7)

From the above shape of the data set it is clear that the user has rated 103 books

### Get top 10 recommendations for above given userID from the books not already rated by that user

In [43]:
non_rated=books[~books['ISBN'].isin(user_full_info['ISBN'])]
sorted_predictions_df=pd.DataFrame(sorted_predictions).reset_index()
sorted_predictions_df

ISBN         2
0      0316666343  1.015397
1      059035342X  0.778665
2      0345350499  0.697309
3      0440214041  0.665439
4      044021145X  0.663549
...           ...       ...
66567  0380709562 -0.162346
66568  0553213164 -0.173301
66569  0440219078 -0.174497
66570  0807508527 -0.204146
66571  0743235150 -0.209452

[66572 rows x 2 columns]

In [44]:
recommendations=pd.merge(non_rated,sorted_predictions_df,on='ISBN').rename(columns = {2: 'Predictions'})

In [45]:
recommendations.sort_values('Predictions', ascending = False).head(10)

ISBN                                          bookTitle  \
237    0316666343                          The Lovely Bones: A Novel   
1164   0345350499                                The Mists of Avalon   
1322   0440214041                                  The Pelican Brief   
252    044021145X                                           The Firm   
292    0312195516                The Red Tent (Bestselling Backlist)   
10050  0345318862      Golem in the Gears (Xanth Novels (Paperback))   
2545   0345313151  Bearing an Hourglass (Incarnations of Immortal...   
3287   0380752891           Man from Mundania (Xanth Trilogy, No 12)   
19741  051511605X                                    Undue Influence   
4570   043936213X     Harry Potter and the Sorcerer's Stone (Book 1)   

                  bookAuthor  yearOfPublication  \
237             Alice Sebold               2002   
1164   MARION ZIMMER BRADLEY               1987   
1322            John Grisham               1993   
252             John Grisham               1992   
292            Anita Diamant               1998   
10050          PIERS ANTHONY               1986   
2545           Piers Anthony               1991   
3287           Piers Anthony               1990   
19741    Steven Paul Martini               1995   
4570           J. K. Rowling               2001   

                           publisher  Predictions  
237                    Little, Brown     1.015397  
1164                         Del Rey     0.697309  
1322                            Dell     0.665439  
252     Bantam Dell Publishing Group     0.663549  
292                      Picador USA     0.642840  
10050                        Del Rey     0.639465  
2545                   Del Rey Books     0.631446  
3287   Harper Mass Market Paperbacks     0.629143  
19741                     Jove Books     0.617955  
4570                      Scholastic     0.614288